<a href="https://colab.research.google.com/github/lvllvl/writing_sample/blob/main/data_dataiku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code / Data Preparation

In [1]:
# Import libraries required to manipulate / format data
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
# Open each CSV file, store the contents in a data frame, a pandas data structure
# that stores tabular data.
df_accounts = pd.read_csv( '/content/drive/MyDrive/TW_assessment/accounts_agents.csv' )
df_contracts = pd.read_csv( '/content/drive/MyDrive/TW_assessment/contracts.csv' )

## Data cleansing


In [3]:
# Convert date-related columns to datetime format
df_contracts['paymentDate'] = pd.to_datetime( df_contracts['paymentDate'], errors='coerce')
df_contracts['closingDate'] = pd.to_datetime( df_contracts['closingDate'], errors='coerce' )

In [4]:
# Create new column to track number of days between close date and payment date, i.e., track accounts with late payments ( >= 45 days = late payment )
df_contracts['Days btwn Close & Payment'] = df_contracts.apply( lambda row: row['paymentDate'] - row['closingDate'] , axis=1 )
df_contracts

,closingDate,contractSize,contractLength,contractID,paymentDate,Days btwn Close & Payment
0,2013-01-01,70.0,2,DKU-87jss3-0001,2013-04-13,102 days
1,2013-01-02,55.0,2,DKU-cwcx12-0001,2013-04-15,103 days
2,2013-01-02,95.0,1,DKU-xk6z56-0001,2013-05-11,129 days
3,2013-01-02,50.0,1,DKU-thkqgj-0001,2013-04-08,96 days
4,2013-01-02,50.0,1,DKU-ro3guq-0001,2013-03-16,73 days
...,...,...,...,...,...,...
6087,2018-12-09,95.0,1,DKU-pt55hi-0003,NaT,NaT
6088,2018-12-10,410.0,1,DKU-8dxl4l-0003,NaT,NaT
6089,2018-12-11,100.0,2,DKU-xqvd0t-0002,NaT,NaT
6090,2018-12-11,155.0,3,DKU-h72m70-0002,NaT,NaT


In [5]:
# Continue to alter the `Days btwn Close & Payment` column to be of type integer,
# this is done so that we can easily use comparison operators against this column (e.g., >, <, <=, etc.).
df_contracts[ 'Days btwn Close & Payment' ] = (( df_contracts['Days btwn Close & Payment'].dt.total_seconds() / 60 ) / 60) / 24 
df_contracts

,closingDate,contractSize,contractLength,contractID,paymentDate,Days btwn Close & Payment
0,2013-01-01,70.0,2,DKU-87jss3-0001,2013-04-13,102.0
1,2013-01-02,55.0,2,DKU-cwcx12-0001,2013-04-15,103.0
2,2013-01-02,95.0,1,DKU-xk6z56-0001,2013-05-11,129.0
3,2013-01-02,50.0,1,DKU-thkqgj-0001,2013-04-08,96.0
4,2013-01-02,50.0,1,DKU-ro3guq-0001,2013-03-16,73.0
...,...,...,...,...,...,...
6087,2018-12-09,95.0,1,DKU-pt55hi-0003,NaT,NaN
6088,2018-12-10,410.0,1,DKU-8dxl4l-0003,NaT,NaN
6089,2018-12-11,100.0,2,DKU-xqvd0t-0002,NaT,NaN
6090,2018-12-11,155.0,3,DKU-h72m70-0002,NaT,NaN


In [6]:
# Create a new boolean (T/F) column. Using the `Days btwn Close & Payment` column
# create a new column that identifies contracts with late payments, i.e., rows with late payments == True.
df_contracts['Late Payment Bool'] = df_contracts.apply( lambda row: True if (row['Days btwn Close & Payment'] >= 45) else False, axis=1 )
df_contracts

,closingDate,contractSize,contractLength,contractID,paymentDate,Days btwn Close & Payment,Late Payment Bool
0,2013-01-01,70.0,2,DKU-87jss3-0001,2013-04-13,102.0,True
1,2013-01-02,55.0,2,DKU-cwcx12-0001,2013-04-15,103.0,True
2,2013-01-02,95.0,1,DKU-xk6z56-0001,2013-05-11,129.0,True
3,2013-01-02,50.0,1,DKU-thkqgj-0001,2013-04-08,96.0,True
4,2013-01-02,50.0,1,DKU-ro3guq-0001,2013-03-16,73.0,True
...,...,...,...,...,...,...,...
6087,2018-12-09,95.0,1,DKU-pt55hi-0003,NaT,NaN,False
6088,2018-12-10,410.0,1,DKU-8dxl4l-0003,NaT,NaN,False
6089,2018-12-11,100.0,2,DKU-xqvd0t-0002,NaT,NaN,False
6090,2018-12-11,155.0,3,DKU-h72m70-0002,NaT,NaN,False


In [7]:
# Double check to see if all contractIDs are unique.
len( df_contracts[ 'contractID' ].unique() ) == 6092

True

In [8]:
# Check if all accountIDs are uniquly represented in df_accounts
len( df_accounts[ 'accountID' ].unique() ) == 3000

True

In [9]:
# Use a dictionary data structure aggregate relevant info from df_accounts.
# This information will be combined with df_contracts.
accounts_dict = defaultdict(list)

for idx, rows in df_accounts.iterrows():
    accounts_dict[ rows['accountID' ] ].append( rows[ 'sales_agent' ] )
    accounts_dict[ rows['accountID' ] ].append( rows[ 'region' ] )
    accounts_dict[ rows['accountID' ] ].append( rows[ 'partnerInvolved' ] )

#print( accounts_dict ) 


## Combining the Datatables 


In [10]:
# add column for sales agent - based on df_accounts[ 'accountID' ] == df_contracts['contractID' ] 
df_contracts['Sales Agent'] = df_contracts.apply( lambda row: accounts_dict[ row['contractID'][4:-5] ][0], axis=1 )
#df_contracts

In [11]:
# add column for region- based on df_accounts[ 'accountID' ] == df_contracts['contractID' ] 
df_contracts['Region'] = df_contracts.apply( lambda row: accounts_dict[ row['contractID'][4:-5] ][1], axis=1 )
#df_contracts

In [12]:
# add column for Partner Involved- based on df_accounts[ 'accountID' ] == df_contracts['contractID' ] 
df_contracts['Partner Involved'] = df_contracts.apply( lambda row: accounts_dict[ row['contractID'][4:-5] ][2], axis=1 )
df_contracts


,closingDate,contractSize,contractLength,contractID,paymentDate,Days btwn Close & Payment,Late Payment Bool,Sales Agent,Region,Partner Involved
0,2013-01-01,70.0,2,DKU-87jss3-0001,2013-04-13,102.0,True,Cullen Lucero,APAC,No
1,2013-01-02,55.0,2,DKU-cwcx12-0001,2013-04-15,103.0,True,Caden Church,APAC,No
2,2013-01-02,95.0,1,DKU-xk6z56-0001,2013-05-11,129.0,True,Fernanda Guerrero,North America,No
3,2013-01-02,50.0,1,DKU-thkqgj-0001,2013-04-08,96.0,True,Kaden Mullen,North America,No
4,2013-01-02,50.0,1,DKU-ro3guq-0001,2013-03-16,73.0,True,Camille Romero,EMEA,No
...,...,...,...,...,...,...,...,...,...,...
6087,2018-12-09,95.0,1,DKU-pt55hi-0003,NaT,NaN,False,Guillermo Key,EMEA,No
6088,2018-12-10,410.0,1,DKU-8dxl4l-0003,NaT,NaN,False,Dale Bailey,North America,No
6089,2018-12-11,100.0,2,DKU-xqvd0t-0002,NaT,NaN,False,Victor Newman,Latin America,No
6090,2018-12-11,155.0,3,DKU-h72m70-0002,NaT,NaN,False,Wade Bean,EMEA,No


## Retrieve the average contract size with late payments

### 2018 Regional Contract Size Averages
For contracts with late payment(s)

In [13]:
# Using the existing df_contracts table, create a new table that meets the criteria

## Region ==  APAC
df_2018_APAC = df_contracts.query( "`Region` == 'APAC' and `Late Payment Bool` == True and `closingDate` >= '2018-01-01' and `closingDate` <= '2018-12-31'" )
APAC_Average18 = df_2018_APAC[ 'contractSize' ].sum() / len( df_2018_APAC.index )

## Region == North America 
df_2018_NAmerica = df_contracts.query( "`Region` == 'North America' and `Late Payment Bool` == True and `closingDate` >= '2018-01-01' and `closingDate` <= '2018-12-31'" )
NAmerica_Average18 = df_2018_NAmerica[ 'contractSize' ].sum() / len( df_2018_NAmerica.index )

## Region == EMEA  
df_2018_EMEA = df_contracts.query( "`Region` == 'EMEA' and `Late Payment Bool` == True and `closingDate` >= '2018-01-01' and `closingDate` <= '2018-12-31'" )
EMEA_Average18 = df_2018_EMEA[ 'contractSize' ].sum() / len( df_2018_EMEA.index )

## Region == Africa 
df_2018_Africa = df_contracts.query( "`Region` == 'Africa' and `Late Payment Bool` == True and `closingDate` >= '2018-01-01' and `closingDate` <= '2018-12-31'" )
Africa_Average18 = df_2018_Africa[ 'contractSize' ].sum() / len( df_2018_Africa.index )

## Region == Latin America 
df_2018_LatAm = df_contracts.query( "`Region` == 'Latin America' and `Late Payment Bool` == True and `closingDate` >= '2018-01-01' and `closingDate` <= '2018-12-31'" )
LatAm_Average18 = df_2018_LatAm[ 'contractSize' ].sum() / len( df_2018_LatAm.index )

### 2017 Regional Contract Size Averages 
For contracts with late payment(s)

In [14]:
# Using the existing df_contracts table, create a new table that meets the criteria

## Region ==  APAC
df_2017_APAC = df_contracts.query( "`Region` == 'APAC' and `Late Payment Bool` == True and `closingDate` >= '2017-01-01' and `closingDate` <= '2017-12-31'" )
APAC_Average17 = df_2017_APAC[ 'contractSize' ].sum() / len( df_2017_APAC.index )

## Region == North America 
df_2017_NAmerica = df_contracts.query( "`Region` == 'North America' and `Late Payment Bool` == True and `closingDate` >= '2017-01-01' and `closingDate` <= '2017-12-31'" )
NAmerica_Average17 = df_2017_NAmerica[ 'contractSize' ].sum() / len( df_2017_NAmerica.index )

## Region == EMEA  
df_2017_EMEA = df_contracts.query( "`Region` == 'EMEA' and `Late Payment Bool` == True and `closingDate` >= '2017-01-01' and `closingDate` <= '2017-12-31'" )
EMEA_Average17 = df_2017_EMEA[ 'contractSize' ].sum() / len( df_2017_EMEA.index )

## Region == Africa 
df_2017_Africa = df_contracts.query( "`Region` == 'Africa' and `Late Payment Bool` == True and `closingDate` >= '2017-01-01' and `closingDate` <= '2017-12-31'" )
Africa_Average17 = df_2017_Africa[ 'contractSize' ].sum() / len( df_2017_Africa.index )

## Region == Latin America 
df_2017_LatAm = df_contracts.query( "`Region` == 'Latin America' and `Late Payment Bool` == True and `closingDate` >= '2017-01-01' and `closingDate` <= '2017-12-31'" )
LatAm_Average17 = df_2017_LatAm[ 'contractSize' ].sum() / len( df_2017_LatAm.index )

### 2016 Regional Contract Size Averages 
For contracts with late payment(s)

In [15]:
# Using the existing df_contracts table, create a new table that meets the criteria

## Region ==  APAC
df_2016_APAC = df_contracts.query( "`Region` == 'APAC' and `Late Payment Bool` == True and `closingDate` >= '2016-01-01' and `closingDate` <= '2016-12-31'" )
APAC_Average16 = df_2016_APAC[ 'contractSize' ].sum() / len( df_2016_APAC.index )

## Region == North America 
df_2016_NAmerica = df_contracts.query( "`Region` == 'North America' and `Late Payment Bool` == True and `closingDate` >= '2016-01-01' and `closingDate` <= '2016-12-31'" )
NAmerica_Average16 = df_2016_NAmerica[ 'contractSize' ].sum() / len( df_2016_NAmerica.index )

## Region == EMEA  
df_2016_EMEA = df_contracts.query( "`Region` == 'EMEA' and `Late Payment Bool` == True and `closingDate` >= '2016-01-01' and `closingDate` <= '2016-12-31'" )
EMEA_Average16 = df_2016_EMEA[ 'contractSize' ].sum() / len( df_2016_EMEA.index )

## Region == Africa 
df_2016_Africa = df_contracts.query( "`Region` == 'Africa' and `Late Payment Bool` == True and `closingDate` >= '2016-01-01' and `closingDate` <= '2016-12-31'" )
Africa_Average16 = df_2016_Africa[ 'contractSize' ].sum() / len( df_2016_Africa.index )

## Region == Latin America 
df_2016_LatAm = df_contracts.query( "`Region` == 'Latin America' and `Late Payment Bool` == True and `closingDate` >= '2016-01-01' and `closingDate` <= '2016-12-31'" )
LatAm_Average16 = df_2016_LatAm[ 'contractSize' ].sum() / len( df_2016_LatAm.index )

# Question 1: By region
- Identify average size of contracts with late payments in 2018
- Identify average size of contracts with late payments in 2017
- Identify average size of contracts with late payments in 2016

How do these numbers all compare to each other?

In [16]:
# Create a table that incorporates all the averages from every region, for each year (2018, 2017, 2016)
details = {
    'Region' : ['APAC', 'North America', 'EMEA', 'Africa', 'Latin America' ],
    '2018 Averages' : [ APAC_Average18, NAmerica_Average18, EMEA_Average18, Africa_Average18, LatAm_Average18 ],
    '2017 Averages' : [ APAC_Average17, NAmerica_Average17, EMEA_Average17, Africa_Average17, LatAm_Average17 ],
    '2016 Averages' : [ APAC_Average16, NAmerica_Average16, EMEA_Average16, Africa_Average16, LatAm_Average16 ]
}
df_question1 = pd.DataFrame( details )
df_question1

,Region,2018 Averages,2017 Averages,2016 Averages
0,APAC,182.297101,151.531034,133.055118
1,North America,192.702929,166.813449,153.645977
2,EMEA,181.019231,165.818000,150.189573
3,Africa,136.489362,131.494505,113.209877
4,Latin America,151.717391,124.872093,123.618421


# Question 2: 

For contracts closed in 2018:
- What is the current ( where "today" is 12-11-2018) average size of contracts still to be collected that are more than 45 days past close?
- Is there a relationship between: 
    * These contracts and, 
    * Whether a partner was involved in bringing the customer to the vendor, or the region in which the customer operates? 

In [17]:
df_contracts

,closingDate,contractSize,contractLength,contractID,paymentDate,Days btwn Close & Payment,Late Payment Bool,Sales Agent,Region,Partner Involved
0,2013-01-01,70.0,2,DKU-87jss3-0001,2013-04-13,102.0,True,Cullen Lucero,APAC,No
1,2013-01-02,55.0,2,DKU-cwcx12-0001,2013-04-15,103.0,True,Caden Church,APAC,No
2,2013-01-02,95.0,1,DKU-xk6z56-0001,2013-05-11,129.0,True,Fernanda Guerrero,North America,No
3,2013-01-02,50.0,1,DKU-thkqgj-0001,2013-04-08,96.0,True,Kaden Mullen,North America,No
4,2013-01-02,50.0,1,DKU-ro3guq-0001,2013-03-16,73.0,True,Camille Romero,EMEA,No
...,...,...,...,...,...,...,...,...,...,...
6087,2018-12-09,95.0,1,DKU-pt55hi-0003,NaT,NaN,False,Guillermo Key,EMEA,No
6088,2018-12-10,410.0,1,DKU-8dxl4l-0003,NaT,NaN,False,Dale Bailey,North America,No
6089,2018-12-11,100.0,2,DKU-xqvd0t-0002,NaT,NaN,False,Victor Newman,Latin America,No
6090,2018-12-11,155.0,3,DKU-h72m70-0002,NaT,NaN,False,Wade Bean,EMEA,No


In [18]:
df_Q2_closed18 = df_contracts.query( '`closingDate` >= "2018-01-01" and `closingDate` <= "2018-12-11" and paymentDate == "NaT"' )
ave_contract_size = df_Q2_closed18[ 'contractSize' ].sum() / len( df_Q2_closed18.index )
df_Q2_closed18

,closingDate,contractSize,contractLength,contractID,paymentDate,Days btwn Close & Payment,Late Payment Bool,Sales Agent,Region,Partner Involved
5753,2018-08-07,193.0,1,DKU-7s0qsy-0004,NaT,NaN,False,Kaden Mullen,North America,Yes
5761,2018-08-11,356.0,3,DKU-r1y4f0-0003,NaT,NaN,False,Jace Carrillo,North America,No
5764,2018-08-11,92.0,2,DKU-8ad4ec-0002,NaT,NaN,False,Kamari Ayers,North America,No
5799,2018-08-22,222.0,3,DKU-5cee9x-0003,NaT,NaN,False,Charlie Maynard,EMEA,Yes
5803,2018-08-23,75.0,1,DKU-o5fxbu-0002,NaT,NaN,False,Kailee Blevins,APAC,No
...,...,...,...,...,...,...,...,...,...,...
6087,2018-12-09,95.0,1,DKU-pt55hi-0003,NaT,NaN,False,Guillermo Key,EMEA,No
6088,2018-12-10,410.0,1,DKU-8dxl4l-0003,NaT,NaN,False,Dale Bailey,North America,No
6089,2018-12-11,100.0,2,DKU-xqvd0t-0002,NaT,NaN,False,Victor Newman,Latin America,No
6090,2018-12-11,155.0,3,DKU-h72m70-0002,NaT,NaN,False,Wade Bean,EMEA,No


In [19]:
print( "The average contract where payments is still to be collected is: ", ave_contract_size ) 

The average contract where payments is still to be collected is:  179.748031496063


In [20]:
df_partner_not_involved = df_contracts.query( '`closingDate` >= "2018-01-01" and `closingDate` <= "2018-12-11" and paymentDate == "NaT" and `Partner Involved` == "No"' )
sum_partner_not_involved = len( df_partner_not_involved.index ) 
sum_partner_not_involved

201

In [21]:
df_partner_involved = df_contracts.query( '`closingDate` >= "2018-01-01" and `closingDate` <= "2018-12-11" and paymentDate == "NaT" and `Partner Involved` == "Yes"' )
sum_partner_involved = len( df_partner_involved.index ) 
sum_partner_involved

53

In [22]:
# Create a table that incorporates all the averages from every region, for each year (2018, 2017, 2016)
details = {
    'No Partner Involved' : [ sum_partner_not_involved  ],
    'Partner Involved' : [ sum_partner_involved ],
}
df_question2 = pd.DataFrame( details )
df_question2

,No Partner Involved,Partner Involved
0,201,53


In [23]:
# Using the existing df_Q2_closed18 table, create a new table that meets the criteria

## Region ==  APAC
df_2018_APAC = df_Q2_closed18.query( '`Region` == "APAC" and `Partner Involved` == "Yes"' )
APAC_YES_Average18 = len( df_2018_APAC.index )
df_2018_APAC = df_Q2_closed18.query( '`Region` == "APAC" and `Partner Involved` == "No"' )
APAC_NO_Average18 = len( df_2018_APAC.index )

## Region == North America 
df_2018_NAmerica = df_Q2_closed18.query( '`Region` == "North America" and `Partner Involved` == "Yes"' )
NAmerica_YES_Average18 =  len( df_2018_NAmerica.index )
df_2018_NAmerica = df_Q2_closed18.query( '`Region` == "North America" and `Partner Involved` == "No"' )
NAmerica_NO_Average18 =  len( df_2018_NAmerica.index )

## Region == EMEA  
df_2018_EMEA = df_Q2_closed18.query( '`Region` == "EMEA" and `Partner Involved` == "Yes"' )
EMEA_YES_Average18 = len( df_2018_EMEA.index )
df_2018_EMEA = df_Q2_closed18.query( '`Region` == "EMEA" and `Partner Involved` == "No"' )
EMEA_NO_Average18 = len( df_2018_EMEA.index )

## Region == Africa 
df_2018_Africa = df_Q2_closed18.query( '`Region` == "Africa" and `Partner Involved` == "Yes"' )
Africa_YES_Average18 = len( df_2018_Africa.index )
df_2018_Africa = df_Q2_closed18.query( '`Region` == "Africa" and `Partner Involved` == "No"' )
Africa_NO_Average18 = len( df_2018_Africa.index )

## Region == Latin America 
df_2018_LatAm = df_Q2_closed18.query( '`Region` == "Latin America" and `Partner Involved` == "Yes"' )
LatAm_YES_Average18 = len( df_2018_LatAm.index )
df_2018_LatAm = df_Q2_closed18.query( '`Region` == "Latin America" and `Partner Involved` == "No"' )
LatAm_NO_Average18 = len( df_2018_LatAm.index )

In [24]:
# Create a table that incorporates all the averages from every region, for each year (2018, 2017, 2016)
details = {
    'Region' : ['APAC', 'North America', 'EMEA', 'Africa', 'Latin America' ],
    'Partner Involved' : [ APAC_YES_Average18, NAmerica_YES_Average18, EMEA_YES_Average18, Africa_YES_Average18, LatAm_YES_Average18 ],
    'Partner NOT Involved' : [ APAC_NO_Average18, NAmerica_NO_Average18, EMEA_NO_Average18, Africa_NO_Average18, LatAm_NO_Average18 ]
}
df_question2a = pd.DataFrame( details )
df_question2a

,Region,Partner Involved,Partner NOT Involved
0,APAC,8,35
1,North America,17,67
2,EMEA,20,75
3,Africa,4,14
4,Latin America,4,10


In [25]:
df_question2

,No Partner Involved,Partner Involved
0,201,53
